In [50]:
import os
import json
import numpy as np
import pandas as pd
#import socket
import requests
#from time import sleep
#from multiprocessing import Pool, cpu_count
from datetime import datetime, timedelta
from progressbar import ProgressBar
from pymongo import MongoClient

In [5]:
key = os.environ['DARK_SKY_KEY']
lat = '40.9310'
lon = '-78.9572'

In [115]:
''' sources:
http://www.groundhog.org/about/history/
http://www.groundhog.org/fileadmin/user_upload/siteImages/pdf/Groundhog_Day_Historical_Predictions.pdf
'''
early_spring_years = ['1950', '1970', '1975', '1983', '1986', '1988', '1990', 
                      '1995', '1997', '1999', '2007', '2011', '2013', '2016']

In [116]:
def unix_to_datetime(unix_time):
    '''
    Convert Unix time (seconds since 1970-01-01 00:00:00) to datetime
    '''
    orig = datetime(1970, 1, 1, 0, 0, 0, 0)
    new_dt = orig + timedelta(seconds=unix_time)
    return new_dt

In [111]:
years = map(str, np.arange(1944, 2018))

for year in years:
    
    time = '{}-02-02T00:00:00'.format(year)
    url = url = 'https://api.darksky.net/forecast/{}/{},{},{}'.format(key, lat, lon, time)
    try:
        response = requests.get(url)
    except:
        continue
    
    if response.status_code == 200:
        forecast = response.json()
        try:
            print "found data for {}".format(unix_to_datetime(forecast['daily']['data'][0]['time']))
        except:
            continue
    else:
        print "earliest year available: {}".format(int(year) + 1)
        break

earliest year available: 1902
